In [40]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from helper import clean_column_names

In [41]:
def clean_brady_table(df):
    df.columns = [col.lstrip('\'') for col in df.columns]

    df = df.applymap(lambda x: x.lstrip('\'') if isinstance(x, str) else x)
    
    df = df.applymap(lambda x: x.lower().strip() if isinstance(x, str) else x)


    df.loc[:, "last_name"] = (df.last_name
                              .str.replace(r"\'RULE(.+)", "", regex=True)
                              .str.replace(r"^\'$", "", regex=True)
    )

    df = df[~((df.last_name.fillna("") == ""))]

    df.loc[:, "tracking_id"] = (df.tracking_id
                                .str
                                .lower()
                                .str.strip()
                                .str.replace(r"\s+", "", regex=True)
                                .str.replace(r"\'", "", regex=True)
    )

    df.loc[:, "allegation"] = (df.allegation_rule
                               .str.cat(df.allegation_paragraph, sep=" ")
    )

    df.loc[:, "allegation"] = (df.allegation
                               .str.lower()
                               .str.strip()
                               .str.replace(r"\'", "", regex=True)
    )

    df = df.drop(columns=["allegation_rule", "allegation_paragraph"])

    df.loc[:, "allegation_desc"] = (df.allegation_desc.str
                                    .lower()
                                    .str.strip()
                                    .str.replace(r"\'$", "", regex=True)
                                    .str.replace(r"^$", "missing", regex=True)
    )



    return df 

def read_brady_tbl():
    df = pd.read_csv("../../analysis/data/output/brady_with_gists.csv")
    return df 

df = read_brady_tbl()

df.loc[:, "tracking_id"] = df.tracking_id.str.replace(r"\-(\w{1})$", "", regex=True)

dv_ids = df.tracking_id.tolist()

df

,tracking_id,incident_type,investigation_status,accused_employee_id,first_name,last_name,allegation_number,allegation_desc,disposition,year,allegation,tracking_id_list
0,2021-0737,public initiated,completed,30307.0,anna,mcelveen,30580,mississippi code title 97-3- 7 (3) domestic vi...,sustained - rui resign,2021,rule 2: moral conduct paragraph 01 - adherence...,NaN
1,2020-0063,public initiated,completed,25966.0,mark,lynch,28395,nopd policy: chapter 42.4 - domestic violence;...,exonerated,2020,rule 4: perf of duty paragraph 04 - neglect of...,NaN
2,2018-0267,public initiated,completed,8913.0,jeffrey,vappie ii,25045,nopd policy: chapter 42.4 - domestic violence,unfounded,2018,rule 4: perf of duty paragraph 04 - neglect of...,NaN
3,2018-0267,public initiated,completed,18876.0,lawrence,weathersby,25046,nopd policy: chapter 42.4 domestic violence,unfounded,2018,rule 4: perf of duty paragraph 04 - neglect of...,NaN
4,2020-0067,public initiated,forwarded,30450.0,ridge,petes,28765,nopd policy: chapter 42.4 - domestic violence,unfounded,2020,rule 4: perf of duty paragraph 04 - neglect of...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
442,2018-0127,public initiated,completed,19456.0,sebastian,trigo,24728,missing,unfounded,2018,rule 3: prof conduct paragraph 01 - profession...,2018-0127
443,2018-0127,public initiated,completed,29836.0,nicholas,wallis,24729,missing,unfounded,2018,rule 3: prof conduct paragraph 01 - profession...,2018-0127
444,2018-0085,rank initiated,completed,6864.0,kevin,williams,24779,paragraph b: any employee with supervisory res...,negotiated settlement,2018,rule 4: perf of duty paragraph 04 - neglect of...,2018-0085
445,2018-0084,public initiated,completed,29281.0,justin,orlansky,24671,missing,nfim case,2018,no allegation assigned at this time,2018-0084


In [42]:
dv_ids = df.tracking_id.tolist()

### clean data nola gov tbl ### 
def clean_gov_table(df):
    df = df.applymap(lambda x: x.lower().strip() if isinstance(x, str) else x)

    df.loc[:, "tracking_id"] = (df["complaint_tracking_number"].str
                                .lower()
                                .str.strip()
                                .str.replace(r"\s+", "", regex=True)
                                .str.replace(r"\'", "", regex=True)
                                .str.replace(r"-(\w{1})$", "", regex=True)
    )


    return df 

def read_gov_tbl():
    df = pd.read_csv("../../data/input/data_nola_gov.csv")
    return df 

def extract_year(df):
    year = df.tracking_id.str.extract(r"^(\w{4})")

    df.loc[:, "year"] = year[0].astype(str)
    return df


gov_tbl = read_gov_tbl()
gov_tbl = gov_tbl.pipe(clean_column_names).pipe(clean_gov_table).pipe(extract_year)

gov_tbl = gov_tbl[gov_tbl.tracking_id.isin(dv_ids)]

gov_tbl.tracking_id.nunique()




/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_31037/1447442045.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower().strip() if isinstance(x, str) else x)


223